<a href="https://colab.research.google.com/github/Yuns-u/Google_platstore_rating-prediction/blob/main/Google_playstore_rating_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 분석 대상: Google Play Store의 application



## 데이터 선정 이유 및 문제 정의
구글 플레이스토어는 국내 94%의 스마트폰 이용자들이 사용하고 있는 스토어이기 때문에 구글 플레이스토어에 앱을 런칭한다면 높은 평점(rating)과 긍정적인 리뷰들을 많이 가지고 있는 것이 유리할 것이다.

해당 데이터는 2018년까지 출시된 앱들의 일부로 구글 플레이스토어의 정량적인 지표들로 구성되어 있다. 이 데이터를 전처리를 하며 살펴본 뒤 평점을 예측하는 머신러닝 모델을 만들어보고자 한다.


In [ ]:
# 필요한 라이브러리 불러오기
!pip install category_encoders

In [ ]:
import pandas as pd
import numpy as np

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
#plt.show를 하지 않아도 된다.

# 사이킷런 - 데이터셋 나누기
from sklearn.model_selection import train_test_split

# 사이킷런 - encoders, imputers
from category_encoders import OneHotEncoder
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer

# 정규화해주기
from sklearn.preprocessing import StandardScaler

# 사이킷런 - pipeline
from sklearn.pipeline import make_pipeline

# 사이킷런 - 학습모델
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression


# 사이킷런 - 모델 평가지표
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/googleplaystore.csv')

# EDA

In [ ]:
df.shape

In [ ]:
df.head()

- 'Size'의 기호를 통일시켜 없애기
- 'Installs'의 기호를 없애고 등급으로 나눠주기
- 'Price' 기호 통일해서 없애기

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
#결측치 확인
df.isna().sum()

In [ ]:
#중복값 확인
df.duplicated().value_counts()

In [ ]:
df[df.duplicated()]

In [ ]:
#중복값들을 없애준다. 
df = df[~df.duplicated()]
df.duplicated().value_counts()

In [ ]:
df.columns

# Column별 Data Cleaning

In [ ]:
#각 feature를 산점도로 봐보기
from pandas.plotting import scatter_matrix

scatter_matrix(df, alpha=0.5, figsize=(20, 20), diagonal='kde')
plt.show()

## column: 'App'
Application name, 
어플리케이션의 이름. 'App'은 각각의 어플리케이션을 구분하게 하는 ID의 역할을 한다.

In [ ]:
df['App'].head()

## column: 'Category'

In [ ]:
df['Category'].value_counts()

genre의 분류표

카테고리 안에 장르가 포함된다. 하위 분류로서의 기능을 한다고 볼 수 있다.

'Art & Design', 'Auto & Vehicles', 'Beauty', 'Books & Reference',
       'Business', 'Comics', 'Communication', 'Dating', 'Education',
       'Entertainment', 'Events', 'Finance', 'Food & Drink',
       'Health & Fitness', 'House & Home', 'Libraries & Demo',
       'Lifestyle', 'Adventure', 'Arcade', 'Casual', 'Card', 'Action',
       'Strategy', 'Puzzle', 'Sports', 'Music', 'Word', 'Racing',
       'Simulation', 'Board', 'Trivia', 'Role Playing', 'Educational',
       'Music & Audio', 'Video Players & Editors', 'Medical', 'Social',
       'Shopping', 'Photography', 'Travel & Local', 'Tools',
       'Personalization', 'Productivity', 'Parenting', 'Weather',
       'News & Magazines', 'Maps & Navigation', 'Casino'

In [ ]:
#_AND_를 &으로 바꿔줘도 좋을 듯. (가독성도 좋아질 듯하다)
#안된다.
#df['Category'] = df.loc[df['Category']=='HEALTH_AND_FITNESS'].replace('HEALTH_AND_FITNESS','HEALTH & FITNESS')
#df['Category'] = df.loc[df['Category']=='NEWS_AND_MAGAZINES'].replace('NEWS_AND_MAGAZINES','NEWS & MAGAZINES')
#df['Category'] = df.loc[df['Category']=='TRAVEL_AND_LOCAL'].replace('TRAVEL_AND_LOCAL','TRAVEL & LOCAL')
#df['Category'] = df.loc[df['Category']=='BOOKS_AND_REFERENCE'].replace('BOOKS_AND_REFERENCE','BOOKS & REFERENCE')
#df['Category'] = df.loc[df['Category']=='MAPS_AND_NAVIGATION'].replace('MAPS_AND_NAVIGATION','MAPS & NAVIGATION')
#df['Category'] = df.loc[df['Category']=='FOOD_AND_DRINK'].replace('FOOD_AND_DRINK','FOOD & DRINK ')
#df['Category'] = df.loc[df['Category']=='AUTO_AND_VEHICLES'].replace('AUTO_AND_VEHICLES','AUTO & VEHICLES')
#df['Category'] = df.loc[df['Category']=='LIBRARIES_AND_DEMO'].replace('LIBRARIES_AND_DEMO','LIBRARIES & DEMO')
#df['Category'] = df.loc[df['Category']=='HOUSE_AND_HOME'].replace('HOUSE_AND_HOME','HOUSE & HOME')
#df['Category'] = df.loc[df['Category']=='ART_AND_DESIGN'].replace('ART_AND_DESIGN','ART & DESIGN')

In [ ]:
#카테고리에 1.9로 잘못 써진 게 있는 듯
df.loc[df['Category']=='1.9']

In [ ]:
#위의 값은 카테고리 항이 누락되면서 한칸씩 앞으로 빠진 케이스.
#여기에서 바꿔준다.

df.at[10472,'Category'] = 'LIFESTYLE'
df.at[10472,'Rating'] = 1.9
df.at[10472,'Reviews'] = 19.0	
df.at[10472,'Size'] = '3.0M'
df.at[10472,'Installs'] = '1,000+'
df.at[10472,'Type'] = 'Free'
df.at[10472,'Price'] = 0
df.at[10472,'Content Rating'] = 'Everyone'
df.at[10472,'Genres'] = 'Lifestyle'
df.at[10472,'Last Updated'] = 'February 11, 2018'
df.at[10472,'Current Ver'] = '1.0.19'
df.at[10472,'Android Ver'] = '4.0 and up'

In [ ]:
df['Category'].value_counts().plot(kind='bar')

plt.title('number of apps for categories in google playstore')

In [ ]:
sns.set(rc={'figure.figsize':(20,10)}, font_scale=1.5, style='whitegrid')
ax = sns.boxplot(x="Category",y="Rating",data=df)
labels = ax.set_xticklabels(ax.get_xticklabels(), rotation=45,ha='right')

## column: 'Rating'
Overall user rating of the app (as when scraped)

Play 스토어의 앱 평점 및 별표 1, 2, 3, 4, 5개 리뷰의 개수에 따라 표시되는 막대그래프는 앱의 평가 횟수가 매우 적은 경우가 아니라면 사용자 리뷰의 전체 기간 평균값이 아닌 앱의 현재 품질 평점을 바탕으로 합니다. (구글 플레이 공식 문서)

In [ ]:
#결측치가 가장 많았던 칼럼이다.
#분포를 먼저 살펴보자.

from pylab import rcParams

# rating distibution 
rcParams['figure.figsize'] = 11.7,8.27
g = sns.kdeplot(df.Rating, color="Red", shade = True)
g.set_xlabel("Rating")
g.set_ylabel("Frequency")
plt.title('Distribution of Rating',size = 20)

In [ ]:
#구글 스토어에는 0~5점으로 표현할 수 있는 것으로 알고 있다.
df['Rating'].describe()

In [ ]:
#1.9가 19.0으로 입력된 것으로 보인다.
#구글 플레이스토어 2021년 8월 31일에 확인해보니 2.7점이긴하지만 해당 데이터는 3년 전에 업데이트된 데이터이므로 2018년 데이터임.
#위의 값을 보니 한 칸씩 밀린 것 같아서 여기에서 수정해야겠다. (위에서 수정함)

In [ ]:
#결측치 처리
#여기에서 결측치는 자료가 없거나 출시된지 얼마 되지 않아서 평가가 되지 않았을 가능성이 있다.
#무작정 결측치를 없애면 안될 것.
rating_null = df[df.Rating.isnull()]
rating_null

In [ ]:
df['Rating'] = df['Rating'].fillna(0)

In [ ]:
# rating distibution 
rcParams['figure.figsize'] = 11.7,8.27
g = sns.kdeplot(df.Rating, color="Red", shade = True)
g.set_xlabel("Rating")
g.set_ylabel("Frequency")
plt.title('Distribution of Rating',size = 20)

In [ ]:
df['Rating'].describe()

In [ ]:
#평균적으로 4점대인 것을 보니 인심이 후하다.
#목표 점수를 4점 혹은 그 이상으로 잡아도 좋을 듯.

## column: 'Reviews'

Number of user reviews for the app (as when scraped)
리뷰의 개수.

In [ ]:
df['Reviews']

In [ ]:
df['Reviews'].describe()

In [ ]:
df['Reviews'].isna().sum()

In [ ]:
#리뷰의 dtype가 object이므로 전부 연속형 수치형으로 바꿔준다.
#문자형이 섞여있는 모양.
#df['Reviews'] = df['Reviews'].apply(lambda x: int(x))

In [ ]:
#df['Reviews']를 int형으로 바꾸기
df['Reviews'] = df['Reviews'].astype(int)

In [ ]:
df['Reviews']

In [ ]:
df['Reviews'].describe()

In [ ]:
#descirbe를 하면 float 형태로 나온다.

print('Reviews count: ',df['Reviews'].count())
print('Reviews mean: ',df['Reviews'].mean())
print('Reviews std: ',df['Reviews'].std())
print('Reviews min: ',df['Reviews'].min())
print('Reviews max: ',df['Reviews'].max())
print('Reviews 1Q(25%): ',df['Reviews'].quantile(.25))
print('Reviews 2Q(50%): ',df['Reviews'].quantile(.50)) 
print('Reviews 3Q(75%): ',df['Reviews'].quantile(.75)) 

In [ ]:
#e+###형식으로 출력되지 않도록 할 수 있다.
pd.set_option('display.float_format', lambda x: '%.5f' % x)

df['Reviews'].describe()

In [ ]:
df['Reviews'] = df['Reviews'].apply(lambda x: int(x))

In [ ]:
# reviews distibution 
rcParams['figure.figsize'] = 11.7,8.27
g = sns.kdeplot(df.Reviews, color="Green", shade = True)
g.set_xlabel("Reviews")
g.set_ylabel("Frequency")
plt.title('Distribution of Reveiws',size = 20)

In [ ]:
#박스 플롯이 더 정확할 것 같다. (시도해봤지만 리뷰의 수가 너무 다양하기 때문에 적절하지 않음)
#산점도가 더 정확할 것 같다.
sns.scatterplot(x = 'Rating', y= 'Reviews', data = df)
plt.title('Reviews scatterplot sorted by Rating(0~5)')
plt.show()

In [ ]:
#정성적인 지표인 review를 볼 땨는 4점 이상 정도는 되어야 충분한 표본을 얻을 수 있을 것이다.

## column:'Size'
Size of the app (as when scraped)

In [ ]:
df['Size'].unique()

In [ ]:
#dtype: int64로 나온다. 왜??
df['Size'].value_counts()

In [ ]:
df['Size'].describe()

In [ ]:
#'Varies with device'를 결측치로 바꿔야 수치형으로 바꿀 수 있을 것이다.
# 용량이기 때문에 매출과 큰 상관관계는 없겠지만 결측치에 카테고리별 평균적인 용량(중앙값)을 넣어주면 어떨까 싶다.
#df['Size'] = df['Size'].replace('Varies with device', np.nan)

In [ ]:
#k = 1000 = 10**3
#M = 1000000 = 10**6
#소문자 m과 대문자 K는 없음.

#Cannot compare types 'ndarray(dtype=int64)' and 'str'
#df['Size'] = (df['Size'].replace(r'[kM]', '', regex=True).astype(float) * \
#              df['Size'].str.extract(r'[\d\.]+([KM]+)', expand=False)
#              .fillna(1)
#              .replace(['k','M'], [10**3, 10**6].astype(int)))

In [ ]:
# Convert kbytes to Mbytes
# 1M = 1000k

k_indices = df['Size'].loc[df['Size'].str.contains('k')].index.tolist()

converter = pd.DataFrame(df.loc[k_indices, 'Size'].apply(lambda x: x.strip('k')).astype(float).apply(lambda x: x / 1024).apply(lambda x: round(x, 3)).astype(str))

df.loc[k_indices,'Size'] = converter

In [ ]:
df['Size'] = df['Size'].apply(lambda x: x.strip('M'))
df['Size'] = df['Size'].replace('Varies with device', 13) #전체 사이즈의 중앙값을 넣었다.
df['Size'] = df['Size'].astype(float)

In [ ]:
df.head()

In [ ]:
df['Size'].describe()

In [ ]:
df['Size'].value_counts()
#0으로 된 부분은 평균값으로 넣어줘도 좋을 것 같다.
#카테고리별 평균을 넣어주어도 좋을 것 같다.

## column: 'Installs'
Number of user downloads/installs for the app (as when scraped)

In [ ]:
#dtype: int64은 왜 나오는 건가?
df['Installs'].value_counts()

In [ ]:
#+는 초과를 의미하는 듯.
#그렇다면 0, 0+, 1+의 의미가 모호해진다.
#매우 적은 다운로드 수를 가진 경우 하나로 묶어줄 수 있을 것 같다.
df['Installs'] = df['Installs'].astype(str)
df['Installs'] = df['Installs'].apply(lambda x: x.strip('+'))
df['Installs'] = df['Installs'].apply(lambda x: x.replace(',',''))
df['Installs'] = df['Installs'].replace('Free',np.nan)

In [ ]:
df['Installs'].value_counts()

In [ ]:
df['Installs'] = df['Installs'].astype(float)

In [ ]:
#수치를 더 간단하게 표현할 수 있을 것 같다.
#Sorted_value = sorted(list(df['Installs'].unique()))
#df['Installs'].replace(Sorted_value,range(0,len(Sorted_value),1), inplace = True )

In [ ]:
#df['Installs'].head()
#광고비 계산이 어려우므로 수치로 남겨둔다.

## column: 'Type'
Paid or Free

In [ ]:
df['Type'].value_counts()

In [ ]:
#free는 가격이 0일 것이며 매우 많음(광고를 더 넣는 식으로 수익 창출, 지명도 높히기)
#paid는 유료앱, 더 깔끔한 편이며 구글 플레이스토어의 경우 앱스토어보다 유료앱이 적은 것으로 알고 있다.
#바이너리로 바꿔주면 어떨까? 분류문제로 푼다면 이 타입을 사용해서 유료앱으로 낼 것인지, 무료앱으로 낼 것인지 알 수 있겠지만
#광고비 등에 대한 매출지표가 없기 때문에(유료앱 제외) 분류문제로 풀기에 적합하지 않음.

#또한 price 항목의 값이 0인 것과 tpye의 Free 값과 같다.
#따라서 분류 문제로 풀지 않는 이상 Type은 없어도 되는 칼럼이다.

## column: 'Price'
Price of the app (as when scraped)

In [ ]:
#달러 기호가 없어야한다.
df['Price'].unique()

In [ ]:
df['Price'].value_counts()

In [ ]:
df['Price'] = df['Price'].apply(lambda x: str(x))

In [ ]:
df['Price'].dtype

In [ ]:
df['Price']=df['Price'].apply(lambda x: x.strip('$'))

In [ ]:
df['Price'].value_counts()

In [ ]:
#0인 것과 Free인 개수는 같다.
#결측치는 가격인데 어떻게 알 수 있는 방법이 없다.
#결측치가 없다. 잘 됐다!
df['Price'].isna().sum()

In [ ]:
df['Price']=pd.to_numeric(df['Price'])
df['Price'].hist();

plt.xlabel('Price')
plt.ylabel('Frequency')

In [ ]:
#이상치인가?
#실제로 판매되고 있는 개그성 앱. 이걸 이상치로 봐야할 것인가? 쓸데없는 돈자랑이라는 컨셉에 충실한 가격 책정이며 구매한 사람이 3천명은 되는 듯하다.
temp = df['Price'].apply(lambda x: True if x>100 else False)
df[temp].head()

## column: 'Content Rating'
Age group the app is targeted at - Children / Mature 21+ / Adult

In [ ]:
df['Content Rating'].value_counts()

In [ ]:
#Unrated된 것 확인 후 고쳐주기
df.loc[df['Content Rating']=='Unrated']

In [ ]:
#사진 편집 어플리케이션
df.at[7312,'Content Rating'] = 'Everyone'
#게임에 도움이 되는 툴 정도. 게임 등급과 별개.
df.at[8266,'Content Rating'] = 'Everyone'

In [ ]:
df['Content Rating'].value_counts()

In [ ]:
sns.boxplot(x = 'Content Rating', y= 'Rating', data= df)

In [ ]:
#성인 앱은 표본 수가 작아서 유의미하지 않다.
#10대들을 대상으로 한 앱(사용자: 청소년, 성인)은 rating에서 큰 차이가 없는 것으로 보인다.

In [ ]:
#OrdinalEncoding으로 할 수 있다.
encoder = OrdinalEncoder()
df['Content Rating'] = encoder.fit_transform(df['Content Rating'])
df['Content Rating'].value_counts()


## column: 'Genres'
An app can belong to multiple genres (apart from its main category). For eg, a musical family game will belong to

In [ ]:
df['Genres'].value_counts()

In [ ]:
# ;를 통해서 두 가지 이상으로 여러 장르들을 표현한 것 같다.
#앞 뒤가 각기 다르며 순서가 일정하지 않다, 주장르와 하위장르로 구분할 수 있을 것.
sep = ';'
rest = df['Genres'].apply(lambda x: x.split(sep)[0])
rest.unique()

In [ ]:
rest.value_counts()

In [ ]:
df['Pri_Genres'] = rest
df['Pri_Genres'].head()

In [ ]:
rest.value_counts()

In [ ]:
#값이 하나만 있는 Music & Audio는 music으로 옮겨도 되지 않을까?
#옮겨도 무리 없는 앱.
df.loc[df['Pri_Genres']=='Music & Audio']

In [ ]:
df.at[2142,'Pri_Genres'] = 'Music'

In [ ]:
rest = df['Genres'].apply(lambda x: x.split(sep)[-1])
rest.unique()

In [ ]:
rest.value_counts()

In [ ]:
df['Sub_Genres'] = rest
df['Sub_Genres'].head()

In [ ]:
#Pri_Genres의 분포
df['Pri_Genres'].value_counts().plot(kind="bar")
plt.title('Prime Genres of application from 2018 google playstore')
plt.show()

In [ ]:
#Sub_Genres의 분포
df['Sub_Genres'].value_counts().plot(kind="bar")
plt.title('Sub Genres of application from 2018 google playstore')
plt.show()

In [ ]:
#게임이 아닌 이상 해당 서브장르들을 면밀하게 볼 필요는 없다고 할 수 있다.
#Category 항목과 Genre의 항목이 중복인 것들도 많기 때문이다.
#해당 feature는 특정 Category를 개발하기로 선정한 뒤에 각각의 장르별 특성의 차이가 있는지 구분할 때 사용하는 것이 더 의미있다.

## column: 'Last Updated'


In [ ]:
from datetime import datetime, date
df['Last Updated'] = pd.to_datetime(df['Last Updated'])
df['Last Updated'].head()

In [ ]:
df['Last Updated'].sort_values()

In [ ]:
#최근에 업데이트한 것 = 유지보수를 꾸준히 하는 것이라고 볼 수 있을 듯.
df['Last Updated'].sort_values(ascending=False)

### 2018년 하반기 시점에서 예측을 할 수 있을 것이다.

In [ ]:
temp = pd.to_datetime(df['Last Updated'])

df['Last Updated'] = temp.apply(lambda x: date(2018,8,31) - datetime.date(x))
df['Last Updated']

In [ ]:
df['Last Updated'] = df['Last Updated'].dt.days
df['Last Updated']

## column: 'Current Ver'

In [ ]:
df['Current Ver'].unique()

## column: 'Android Ver'

In [ ]:
df['Android Ver'].unique()

In [ ]:
df['Android Ver'].isna().sum()

In [ ]:
df.loc[df['Android Ver']== None]

In [ ]:
#데이터를 범위로 나타냈는데 복잡하고 지저분하다.
#처음 올라간 버전과 끝의 버전을 의미하는 것으로 보인다.

df['Android_Ver_begin'] = df['Android Ver'].apply(lambda x:str(x).split(' and ')[0].split(' - ')[0])
df['Android_Ver_begin'] = df['Android_Ver_begin'].replace('4.4W','4.4')
df['Android_Ver_end'] = df['Android Ver'].apply(lambda x:str(x).split(' and ')[-1].split(' - ')[-1])

In [ ]:
df['Android_Ver_begin'].unique()

In [ ]:
df['Android_Ver_end'].unique()

In [ ]:
df.head()

# Data Cleaning 이후 EDA, feature engineering 등

## 불필요한 column이 된 Type, Genres, Android Ver 제거하기

In [ ]:
df = df.drop(columns=['App','Type','Genres','Pri_Genres','Sub_Genres','Android Ver'])

## 앱 구매 매출

installs * 앱 가격을 하면 각 앱이 사용자들이 앱을 구매함으로써 얻은 매출을 알 수 있을 것 같음.

광고는 광고비* installs를 하면 될 것인데, 가격을 어떻게 측정할 것인지 파악해야 함.

In [ ]:
df['Profit'] = df['Price'] * df['Installs']
df['Profit'].value_counts()

## column 순서 재정렬하기

In [ ]:
df = df[['Category','Size','Installs','Content Rating','Last Updated','Android_Ver_begin','Android_Ver_end','Current Ver','Profit','Price','Reviews','Rating']]

In [ ]:
df.head()

In [ ]:
df.describe()

## Category별 앱의 수

In [ ]:
df['Category'].value_counts().plot(kind='bar')
plt.title('number of apps for categories in google playstore')

## Rating 분포도

In [ ]:
rcParams['figure.figsize'] = 11.7,8.27
g = sns.kdeplot(df.Rating, color='Red', shade = True)
g.set_xlabel('Rating')
g.set_ylabel('Frequency')

plt.axvline(x = df['Rating'].mean(), color='r', linestyle='--')
plt.axvline(x = df['Rating'].median(), color='b', linestyle='--')

plt.title('Distribution of Rating',size = 20)
plt.legend({'Mean':df['Rating'].mean(),'Median':df['Rating'].median()})

### 기준모델

일단, 0점은 평가가 되지 않은 것으로 최근에 출시되어 평점이 없거나, 인지도나 사용자가 없어서 평점이 없는 것으로 보인다. 

In [ ]:
df.loc[df['Rating']==0]

전체의 약 10% 정도가 평점이 0이다. 일반적인 회사라고 가정한다면 평균보다는 중앙값을 기준모델로 삼는 것이 좋을 것이라 생각한다.

In [ ]:
df.groupby(by='Category').median()

기준모델은 전체 앱의 중앙값인 4.2가 될 것이다.

In [ ]:
base = df['Rating'].median()
base

## Reviews 분포도
이것은 어떻게 해석을 해야하는가?


In [ ]:
df['Reviews'] = pd.to_numeric(df['Reviews'])

df.hist(column='Reviews')

plt.xlabel('Reviews')
plt.ylabel('Frequency')

plt.title('number of apps by Reviews')

## Size 분포도

In [ ]:
df.hist(column='Size')

plt.xlabel('Size')
plt.ylabel('Frequency')

plt.title('number of apps by Size(k)')

## Installs 분포도

In [ ]:
df['Installs'] = pd.to_numeric(df['Installs'])
df['Installs'].hist();

plt.xlabel('No. of Installs')
plt.ylabel('Frequency')

In [ ]:
#크게 두 용량으로 나뉘는 것 같은데, 각각의 용량들에 따라 카테고리가 달라질까?

In [ ]:
df['Installs'].describe()

In [ ]:
low_install = df[df['Installs'] < 8.5]
high_install = df[df['Installs'] >= 8.5]

In [ ]:
low_install_mean = low_install.groupby(by='Category').mean()

In [ ]:
high_install_mean = high_install.groupby(by='Category').mean()

In [ ]:
high_install_mean - low_install_mean
#장르마다 용량의 차이가 있지만 유의미하다고 할 수 없다.
#용량이 크다고해서 더 높은 평점을 받는다고 단정할 수 없으며 장르별로 콘텐츠가 많은 것(용량의 크기가 큰 것)

In [ ]:
from scipy import stats
stats.ttest_ind(high_install_mean, low_install_mean, axis= 0, equal_var = True,
                nan_policy='propagate')

In [ ]:
#p-value를 보니 장르별 정량적 특성들(size, install, profit, price, review, rating)의 차이가 크지 않다.

# 풀고자 하는 문제 : 앱들의 rating 예측하기


## cardinality 확인
cardinality확인은 데이터타입이 object인 것만 진행


In [ ]:
df.describe(exclude='number').T.sort_values(by='unique')

In [ ]:
#App은 ID와 같은 것..굳이 필요하지 않을 지도.

In [ ]:
#Current Ver
df['Current Ver'].value_counts()

In [ ]:
#버전관리를 해야하는 문제가 아니기 때문에 없어도 될 것이다.
df = df.drop(columns=['Current Ver'])

In [ ]:
#Content Rating은 ordinal한 순서로 바꿔줄 수 있다.
df['Content Rating'].value_counts()

In [ ]:
#def new_contentRate(df):
#  df['Content Rating'] = ''
#
#  for index in range(len(df)):
#    level = df['Content Rating'][index]
#
#    if level == 'Everyone':
#      df['Content Rating'][index] = int(0)
#
#    elif level == 'Everyone 10+':
#      df['Content Rating'][index] = int(1)
#
#    elif level == 'Teen':
#      df['Content Rating'][index] = int(2)
#
#    elif level == 'Mature 17+':
#      df['Content Rating'][index] = int(3)
#    
#    elif level == 'Adults only 18+':
#      df['Content Rating'][index] = int(4)
#
#  return df

In [ ]:
#함수 실행
#new_contentRate(df)

In [ ]:
#버전 업데이트와 곤련해서는 사실 그렇게 크게 중요하지 않는 것 같다.
#앱의 유지보수와 관련된 데이터나 벤치마킹에 필요한 데이터이므로 현재의 분석에서 유용하다고 하기 어려울 것 같다.
#분석의 시점에 따라 유용성이 크게 달라질 것이다. 현재 분석의 시점은 앱을 개발하기 전이라고 가정했다.

df = df.drop(columns=['Android_Ver_end','Android_Ver_begin'])

## feature들 간의 상관관계

In [ ]:
df.dtypes
corr = df.apply(lambda x: x.factorize()[0]).corr()
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns,annot=True)

대부분의 feature들과 상관관계는 크지 않는 것으로 보이나 rating과 install이 상대적으로 상관관계가 있는 것처럼 보인다.

## 타겟 특성들의 클래스 비율 살펴보기

In [ ]:
df['Rating'].describe()

In [ ]:
#4.2보다 크면 상대적으로 좋은 평점, 4.2보다 낮으면 상대적으로 낮은 평점이라고 할 수 있다.
#이상적으로 정규화가 되었다면 이라면 보통인 3이 가장 높아야하지만 구글 플레이스토어의 평점이 상대적으로 후한 편이라고 생각된다.
highRating = df.copy()
highRating = highRating.loc[highRating["Rating"] >= 4.2]
highRateNum = highRating.groupby('Category')['Rating'].nunique()
highRateNum.sort_values()

In [ ]:
#높은 rating을 가지면서 적은 개수인 카테고리에 신경을 써주어야한다.
#왜냐하면 성공한 레퍼런스가 적은 앱을 개발한다는 건 레퍼러스가 적다는 것이고, 
#이는 개발자, 기획자, 디자이너가 새로운 혁신들을 내야 한다는 의미이기도 하기 때문이다.
#여기에서는 entertainment 카테고리에 유의해야할 것이다.
#해당 데이터는 1만개 정도 되는 토이 데이터로 데이터 수가 적으므로 이를 간과하고 진행해도 괜찮을 것이다.
#그러나 데이터의 수가 많아지면 차이가 커질 것이기 때문에 더 큰 데이터를 가지고 모델링을 할 때 유의해야할 것이다.

# 모델 만들기

## 인코딩

In [ ]:
# 데이터 분리해주기
target = 'Rating'

df_target = df[target]
df_features = df.drop(target, axis=1)

X_train, X_test, y_train, y_test = train_test_split(df_features,df_target, test_size=0.2 ,random_state=0)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2 ,random_state=0)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_val.shape, y_val.shape

In [ ]:
X_test.shape, y_test.shape

In [ ]:
df.columns

In [ ]:
#특성별로 인코더를 따로 정해줄 필요가 있는 것 같아서 특성의 이름을 모아 리스트로 설정

#onehot을 할 특성
onehot = ['Category']

#ordinal할 특성
ordinal = ['Size','Installs','Content Rating','Last Updated','Profit','Price','Reviews']

In [ ]:
encoder = OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_val = encoder.fit_transform(X_val)
X_test = encoder.fit_transform(X_test)

## 기준모델 성능

기준모델로 삼은 기준은 중앙값이기 때문에 50% 정도의 정확성을 가진다.
즉, 머신러닝 모델들은 50%보다 높아야 기준모델보다 성능이 좋다고 할 수 있다.

## Model Selecting

## 선형회귀: 단순선형회귀, 다중선형회귀, 릿지회귀


### 단순&다중선형회귀


In [ ]:
#기준모델(Rating의 중앙값)
base

In [ ]:
model = LinearRegression()

In [ ]:
#rating에 비교적 영향을 많이 주는 feature들을 추려서 feature 고르기
df_corr = df.corr()
df_corr = df_corr.replace(1, np.NaN).abs() #상관계수가 1인 것들을 결측치로 바꿔주고 각 상관계수를 절댓값으로 바꿔주기
df_corr = df_corr[['Rating']]
df_corr.sort_values(by=['Rating'], ascending = False)

In [ ]:
#Size 하나만 가지고 학습시켜보기
#이건 다중선형회귀가 아닌 단순선형회귀이다.
features1 = ['Size']

X_train1 = X_train[features1]
y_train1 = y_train
X_test1 = X_test[features1]
y_test1 = y_test

model1= LinearRegression()
model1.fit(X_train1, y_train1)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

y_pred1 = model1.predict(X_train1)

mae = mean_absolute_error(y_train1, y_pred1)
print(f'훈련 에러(MAE): {mae}')

mse = mean_squared_error(y_train1, y_pred1)
print(f'훈련 에러(MSE): {mse}')

train1_r2 = r2_score(y_train1, y_pred1)
print(f'훈련 에러(R2): {train1_r2}')

y_pred1 = model1.predict(X_test1)

mae = mean_absolute_error(y_test1, y_pred1)
print(f'테스트 에러(MAE): {mae}')

mse = mean_squared_error(y_test1, y_pred1)
print(f'테스트 에러(MSE): {mse}')

test1_r2 = r2_score(y_test1, y_pred1)
print(f'훈련 에러(R2): {test1_r2}')

In [ ]:
#단순선형회귀는 0.02 정도의 설명령을 가지고 있으므로 너무 낮은 확률을 보여준다.
#단순선형회귀는 사용하지 않는다.

In [ ]:
#다중선형회귀: feature 2개, 선형회귀와 큰 차이 없음.
features2 = ['Size','Last Updated']

X_train2 = X_train[features2]
y_train2 = y_train
X_test2 = X_test[features2]
y_test2 = y_test

model2= LinearRegression()
model2.fit(X_train2, y_train2)

y_pred2 = model2.predict(X_train2)

mae = mean_absolute_error(y_train2, y_pred2)
print(f'훈련 에러(MAE): {mae}')

train2_r2 = r2_score(y_train2, y_pred2)
print(f'훈련 에러(R2): {train1_r2}')

y_pred2 = model2.predict(X_test2)
mae = mean_absolute_error(y_test2, y_pred2)
print(f'테스트 에러(MAE): {mae}')

test2_r2 = r2_score(y_test2, y_pred2)
print(f'훈련 에러(R2): {test2_r2}')

In [ ]:
#다중선형회귀: feature 3개, 단순선형회귀와 큰 차이가 없으며 설명령이 더 향상되지 않는다.
features3 = ['Size','Last Updated']

X_train3 = X_train[features3]
y_train3 = y_train
X_test3 = X_test[features3]
y_test3 = y_test

model3= LinearRegression()
model3.fit(X_train3, y_train3)

y_pred3 = model3.predict(X_train3)

mae = mean_absolute_error(y_train3, y_pred3)
print(f'훈련 에러(MAE): {mae}')

mse = mean_squared_error(y_train3, y_pred3)
print(f'훈련 에러(MSE): {mse}')

train3_r2 = r2_score(y_train3, y_pred3)
print(f'훈련 에러(R2): {train1_r2}')

y_pred3 = model3.predict(X_test3)

mae = mean_absolute_error(y_test3, y_pred3)
print(f'테스트 에러(MAE): {mae}')

mse = mean_squared_error(y_test3, y_pred3)
print(f'테스트 에러(MSE): {mse}')

test3_r2 = r2_score(y_test3, y_pred3)
print(f'훈련 에러(R2): {test3_r2}')

### 릿지회귀

In [ ]:
#릿지 회귀모델의 하이퍼파라미터 튜닝하기
from sklearn.linear_model import RidgeCV

alphas = [0.01, 0.05, 0.1, 0.15,0.2, 1.0, 10.0, 100.0]

ridge = RidgeCV(alphas = alphas, 
                normalize=True, 
                cv=3)

ridge.fit(X_train, y_train)

print('alpha: ', ridge.alpha_)

In [ ]:
model = Ridge(alpha = 0.2, normalize=True)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mae= mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Test MAE: {mae}')
print(f'Test R2: {r2}')

In [ ]:
#릿지회귀로도 거의 설명을 못한다고 볼 수 있다.
#선형회귀모델은 사용할 수 없다.

## 분류모델: Logistic Regression, RandomForest

In [ ]:
#분류를 위해 타겟을 0과 1로 나눠줄 필요가 있을 듯하다.
f = lambda x: 1 if x >= 4.2 else 0
df['Rating'] = df['Rating'].map(f)

In [ ]:
df['Rating'].value_counts()

In [ ]:
target = 'Rating'

df_target = df[target]
df_features = df.drop(target, axis=1)

X_train, X_test, y_train, y_test = train_test_split(df_features,df_target, test_size=0.2 ,random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2 ,random_state=0)

## Logistic Regression

In [ ]:
from sklearn.preprocessing import StandardScaler

pipe = make_pipeline(
    OneHotEncoder(cols='Category'),
    SimpleImputer(strategy='median'),
    LogisticRegression(random_state=2,n_jobs=-1)
)

pipe.fit(X_train, y_train)


print('검증 세트 예측 정확도', pipe.score(X_val, y_val))

In [ ]:
encoder = OneHotEncoder(use_cat_names=True, cols='Category')
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.fit_transform(X_val)

In [ ]:
#1) 하이퍼파라미터 C 조정 :큰 차이가 없다.
#객체 생성 & 모델 훈련

logistic_001 = LogisticRegression(C=0.01, max_iter= 1000).fit(X_train_encoded, y_train)
logistic_01 = LogisticRegression(C=0.1, max_iter= 1000).fit(X_train_encoded, y_train)
logistic_0 = LogisticRegression(max_iter= 1000).fit(X_train_encoded, y_train)
logistic_1 = LogisticRegression(C=1, max_iter= 1000).fit(X_train_encoded, y_train)
logistic_10 = LogisticRegression(C=10, max_iter= 1000).fit(X_train_encoded, y_train)

print('C=0.01 정확도', logistic_001.score(X_val_encoded, y_val))
print('C=0.1 정확도', logistic_01.score(X_val_encoded, y_val))
print('C=0 정확도', logistic_0.score(X_val_encoded, y_val))
print('C=1 정확도', logistic_1.score(X_val_encoded, y_val))
print('C=10 정확도', logistic_10.score(X_val_encoded, y_val))

In [ ]:
#2) 표준화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_val_scaled = scaler.transform(X_val_encoded)

In [ ]:
#객체 생성 & 모델 훈련

logistic_001 = LogisticRegression(C=0.01, max_iter= 1000).fit(X_train_scaled, y_train)
logistic_01 = LogisticRegression(C=0.1, max_iter= 1000).fit(X_train_scaled, y_train)
logistic_0 = LogisticRegression(max_iter= 1000).fit(X_train_scaled, y_train)
logistic_1 = LogisticRegression(C=1, max_iter= 1000).fit(X_train_scaled, y_train)
logistic_10 = LogisticRegression(C=10, max_iter= 1000).fit(X_train_scaled, y_train)

print('C=0.01 정확도', logistic_001.score(X_val_scaled, y_val))
print('C=0.1 정확도', logistic_01.score(X_val_scaled, y_val))
print('C=0 정확도', logistic_0.score(X_val_scaled, y_val))
print('C=1 정확도', logistic_1.score(X_val_scaled, y_val))
print('C=10 정확도', logistic_10.score(X_val_scaled, y_val))

In [ ]:
#원래의 값이 더 낫다.
print('검증 세트 정확도', pipe.score(X_val,y_val))
print('테스트 세트 정확도', pipe.score(X_test,y_test))

## RandomForest

In [ ]:
pipe = make_pipeline(
    OneHotEncoder(),
    SimpleImputer(),
    RandomForestClassifier(n_jobs=-1, random_state=10, oob_score=True)
)

pipe.fit(X_train, y_train)
print('검정정확도: ', pipe.score(X_val, y_val))

y_pred = pipe.predict(X_val)
print('검증 f1 score: ',f1_score(y_val, y_pred))

In [ ]:
#하이퍼 파라미터 조정하기

from sklearn.model_selection import RandomizedSearchCV

parameters = {   
    'randomforestclassifier__max_depth': range(1, 20, 2), 
    'randomforestclassifier__max_features': range(1, 20, 2), 
    'randomforestclassifier__min_samples_leaf' : range(1, 20, 2)
}
    

rf_classifier = RandomizedSearchCV(
    pipe, 
    param_distributions=parameters, 
    n_iter=10, 
    cv=5,
    scoring='accuracy',
    verbose=1,
)

rf_classifier.fit(X_train, y_train);

In [ ]:
# RandomCV 결과 확인
print('Best Parameters: ', rf_classifier.best_params_)
print('MAE: ', -rf_classifier.best_score_)

In [ ]:
# 가장 최고의 모델로 학습 -> validation dataset score
best_pipe = rf_classifier.best_estimator_

y_pred = best_pipe.predict(X_val)
mae = mean_absolute_error(y_val, y_pred)
print(f'검증세트 MAE: {mae}')
print(classification_report(y_val, y_pred))

In [ ]:
y_pred = best_pipe.predict(X_test)

print('\n <test 정확도> \n', accuracy_score(y_test, y_pred))
print('\n <classification metrics> \n', classification_report(y_pred, y_test))
print('\n <f1 score> \n', f1_score(y_pred, y_test))

In [ ]:
#ROC Curve
from sklearn.metrics import roc_curve

y_pred_proba = best_pipe.predict_proba(X_val)[:,1]

fpr, tpr, thresholds = roc_curve(y_val, y_pred_proba)

roc = pd.DataFrame({
    'FPR(Fall-out)': fpr,
    'TPR(Recall)': tpr,
    'Threshold': thresholds
})

plt.scatter(fpr, tpr)
plt.title('ROC curve for RandomForest best pipe')
plt.xlabel('FPR(Fall-out)')
plt.ylabel('TPR(Recall)')

In [ ]:
from sklearn.metrics import roc_auc_score

auc_score = roc_auc_score(y_val, y_pred_proba)
auc_score

## xgboost 


In [ ]:
y_train.value_counts(normalize=True)

In [ ]:
ratio = 0.44448/0.55552
ratio

In [ ]:
from xgboost import XGBClassifier

processor = make_pipeline(
    OrdinalEncoder(), 
    SimpleImputer(strategy='median')
)

X_train_processed = processor.fit_transform(X_train)
X_val_processed = processor.transform(X_val)

eval_set = [(X_train_processed, y_train), 
            (X_val_processed, y_val)]

# XGBoost 분류기를 학습시키기
model = XGBClassifier(n_estimators=1000, verbosity=0, n_jobs=-1, scale_pos_weight=ratio)
model.fit(X_train_processed, y_train, eval_set=eval_set, eval_metric='auc', 
          early_stopping_rounds=10)

In [ ]:
X_test_processed = processor.transform(X_test)
X_val_processed = processor.transform(X_val)

class_index = 1

y_pred_proba = model.predict_proba(X_test_processed)[:, class_index]

print(f'Test AUC for class "{model.classes_[class_index]}":')
print(roc_auc_score(y_test, y_pred_proba))

In [ ]:
y_pred = model.predict(X_val_processed)
print('검증 정확도',accuracy_score(y_val,y_pred))

In [ ]:
y_pred = model.predict(X_test_processed)
print('테스트 정확도',accuracy_score(y_test,y_pred))

In [ ]:
from sklearn.metrics import classification_report
y_test_pred = model.predict(X_test_processed)
print(classification_report(y_test, y_test_pred))

# 가장 설명력이 좋은 것으로 보이는 randomforest bestpipe를 채택한다.

# 머신러닝 모델 해석

In [ ]:
!pip install eli5

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

best_pipe.named_steps

In [ ]:
permuter = PermutationImportance(
    best_pipe.named_steps['randomforestclassifier'],
    scoring = 'accuracy',
    n_iter=5,
    random_state=2
)

In [ ]:
X_test_transformed = best_pipe.named_steps['onehotencoder'].transform(X_test)

permuter.fit(X_test_transformed, y_test);

In [ ]:
feature_names = X_test_transformed.columns.tolist()
pd.Series(permuter.feature_importances_,feature_names).sort_values()

In [ ]:
eli5.show_weights(
    permuter,
    top=None,
    feature_names = feature_names
)

가장 높은 값을 가지는 특성은 Reviews의 개수와 Installs(다운로드 수)라고 할 수 있다.

## 가장 큰 영향을 주는 Reviews에 대해서 PDP 만들기

In [ ]:
!pip install pdpbox

In [ ]:
from pdpbox.pdp import pdp_isolate, pdp_plot
from sklearn.metrics import r2_score

feature = 'Reviews'

isolated = pdp_isolate(
    model = best_pipe,
    dataset = X_train,
    model_features = X_train.columns,
    feature = feature
)

pdp_plot(isolated, feature_name=feature);

In [ ]:
feature = 'Installs'

isolated = pdp_isolate(
    model = best_pipe,
    dataset = X_train,
    model_features = X_train.columns,
    feature = feature
)

pdp_plot(isolated, feature_name=feature);

## SHAP

In [ ]:
!pip install shap

In [ ]:
import shap

In [ ]:
best_pipe

In [ ]:
encoder = OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_val = encoder.fit_transform(X_val)
X_test = encoder.fit_transform(X_test)

In [ ]:
#force plot
shap.initjs();

explainer = shap.TreeExplainer(best_pipe['randomforestclassifier'])
observations = best_pipe['simpleimputer'].transform(X_test)

shap_values = explainer.shap_values(X_train)



shap.force_plot(
    base_value = explainer.expected_value,
    shap_values = shap_values,
    features = X_test.columns
)

In [ ]:
#summary plot
shap_values = explainer.shap_values(observations)
shap.summary_plot(shap_values, X_test, plot_type="bar")

In [ ]:
#feature importance
importances = pd.Series(best_pipe['randomforestclassifier'].feature_importances_, X_train.columns)

n = 13
plt.figure(figsize=(10, n/2))
plt.title(f'Top {n}features')
importances.sort_values()[-n:].plot.barh();